In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import torch
import random
from HP import HawkesProcess

In [17]:
#K = 3
N = 100
D = 1
C = 10
T = 10
mu = torch.rand(C)
A = torch.rand(C, C, D)
basis_fs = [lambda x: torch.exp(- x**2 / 10.)]
s = []
Ts = []

for n in range(N):
    Tn = 3
    Mn = random.randint(10, 20)
    Ts.append(Tn)
    ts = torch.FloatTensor([random.random() * Tn for _ in range(Mn)]).sort()[0]
    cs = torch.FloatTensor([random.randint(0, C - 1) for _ in range(Mn)])
    s.append(torch.stack([ts, cs], -1))

In [18]:
hp = HawkesProcess(mu, A, basis_fs, Ts, s)

In [19]:
lambdat = hp.intensity(t=0.1)

In [20]:
lambdat.shape

torch.Size([100, 10])

In [21]:
lambdas = hp.intensity_in_real_points()

In [22]:
len(lambdas), lambdas[0].shape

(100, torch.Size([10, 16]))

In [23]:
ll = hp.log_likelihood()

In [24]:
ll.shape

torch.Size([100])

In [25]:
ll

tensor([-190.6915, -218.4707, -181.8362, -123.9356, -182.9100, -136.7564,
        -135.6404, -152.5795, -201.5201, -152.1406, -217.2238, -186.3911,
        -222.2392, -230.6924, -243.6796, -128.6820, -207.3017, -134.9239,
        -242.6823, -118.3326, -164.2163, -187.7713, -234.9419, -220.8348,
        -186.5730, -217.7605, -239.6735, -129.6948, -258.8388, -155.9521,
        -152.6347, -179.0357, -173.5359, -165.6416, -214.8890, -249.5168,
        -169.8995, -193.2991, -190.0383, -162.9794, -131.4406, -235.7739,
        -227.3759, -192.5802, -210.5151, -136.8567, -238.4410, -202.0022,
        -218.7082, -206.0655, -228.4102, -240.9238, -221.3288, -218.9867,
        -155.7887, -117.9244, -235.0131, -126.6531, -226.9058, -260.9292,
        -202.0644, -196.5120, -183.0843, -108.7485, -150.0526, -143.6206,
        -202.5307, -167.1011, -202.5936, -154.8176, -149.5382, -170.4017,
        -220.3690, -162.1810, -132.1668, -233.4304, -246.6002, -186.3751,
        -220.7843, -222.3449, -142.540

In [26]:
s = []
Ts = 3

for n in range(N):
    Mn = random.randint(10, 20)
    ts = torch.FloatTensor([random.random() * Ts for _ in range(Mn)]).sort()[0]
    cs = torch.FloatTensor([random.randint(0, C - 1) for _ in range(Mn)])
    s.append(torch.stack([ts, cs], -1))

In [27]:
hp = HawkesProcess(mu, A, basis_fs, Ts, s)

In [28]:
ll = hp.log_likelihood()

In [29]:
ll

tensor([-215.9946, -173.6475, -214.5579, -188.9017, -240.5219, -226.5906,
        -133.9746, -224.9023, -149.6643, -203.3722, -234.6306, -191.5668,
        -138.9627, -218.7311, -249.6678, -173.2766, -199.4632, -235.7372,
        -160.6410, -116.9598, -248.2590, -119.0684, -221.2399, -147.8428,
        -133.3368, -134.0710, -231.9751, -202.7340, -163.6279, -116.3683,
        -219.8740, -237.3951, -139.8130, -112.5899, -166.6431, -181.1646,
        -251.7330, -216.3917, -239.2775, -170.4203, -112.5538, -256.0995,
        -160.6224, -219.9014, -166.8435, -204.0991, -141.8255, -202.2490,
        -134.2833, -167.7646, -196.6648, -208.7373, -139.0931, -183.8744,
        -224.2987, -172.1747, -109.4682, -231.2920, -192.1536, -203.7910,
        -132.8038, -184.4113, -217.5217, -131.2305, -206.9374, -177.0584,
        -121.9180, -147.7027, -141.2010, -152.6486, -111.5692, -249.3198,
        -200.0404, -145.3538, -131.5015, -125.5786, -207.1881, -180.1033,
        -212.2836, -197.6812, -192.693

In [ ]:
z